<a href="https://colab.research.google.com/github/chihchao/application-of-programming/blob/main/web_api_linebot_rich_menu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q flask pyngrok flask_ngrok line-bot-sdk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.9/779.9 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 9.2 MB/s eta 0:00:00


In [10]:
from google.colab import userdata
from linebot import LineBotApi
from linebot.models import (
    RichMenu, RichMenuArea, RichMenuBounds,
    MessageAction, URIAction, DatetimePickerAction
)

line_bot_api = LineBotApi(userdata.get('LINE_CHANNEL_ACCESS_TOKEN'))

rich_menu_to_create = RichMenu(
    size={"width": 800, "height": 540},
    selected=True,
    name="六格選單",
    chat_bar_text="開啟選單",
    areas=[
        RichMenuArea(bounds=RichMenuBounds(x=0, y=0, width=266, height=270),
                     action=URIAction(label='Pixabay', uri='https://pixabay.com/')),
        RichMenuArea(bounds=RichMenuBounds(x=266, y=0, width=267, height=270),
                     action=MessageAction(text="芙莉蓮超好看")),
        RichMenuArea(bounds=RichMenuBounds(x=533, y=0, width=267, height=270),
                     action=MessageAction(text="圖文選單：我要傳位置")),
        RichMenuArea(bounds=RichMenuBounds(x=0, y=270, width=266, height=270),
                     action=MessageAction(text="圖文選單：我要拍照")),
        RichMenuArea(bounds=RichMenuBounds(x=266, y=270, width=267, height=270),
                     action=DatetimePickerAction(label="選擇時間", data="時間", mode="datetime")),
        RichMenuArea(bounds=RichMenuBounds(x=533, y=270, width=267, height=270),
                     action=MessageAction(text="圖文選單：選擇照片")),
    ]
)

# 建立 Rich Menu
rich_menu_id = line_bot_api.create_rich_menu(rich_menu_to_create)
print("Rich Menu ID:", rich_menu_id)

# 上傳圖片（PNG 格式）
with open("/content/05-rich_menu.png", 'rb') as f:
    line_bot_api.set_rich_menu_image(rich_menu_id, 'image/png', f)

# 設為預設選單（可選）
line_bot_api.set_default_rich_menu(rich_menu_id)


<ipython-input-10-f8ccb8e78054>:8: LineBotSdkDeprecatedIn30: Call to deprecated class LineBotApi. (Use v3 class; linebot.v3.<feature>. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api = LineBotApi(userdata.get('LINE_CHANNEL_ACCESS_TOKEN'))
<ipython-input-10-f8ccb8e78054>:32: LineBotSdkDeprecatedIn30: Call to deprecated method create_rich_menu. (Use 'from linebot.v3.messaging import MessagingApi' and 'MessagingApi(...).create_rich_menu(...)' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  rich_menu_id = line_bot_api.create_rich_menu(rich_menu_to_create)


Rich Menu ID: richmenu-bc52662bb2cf036fc0fdf1df560a4ba9


<ipython-input-10-f8ccb8e78054>:37: LineBotSdkDeprecatedIn30: Call to deprecated method set_rich_menu_image. (Use 'from linebot.v3.messaging import MessagingApiBlob' and 'MessagingApiBlob.set_rich_menu_image(...)' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api.set_rich_menu_image(rich_menu_id, 'image/png', f)
<ipython-input-10-f8ccb8e78054>:40: LineBotSdkDeprecatedIn30: Call to deprecated method set_default_rich_menu. (Use 'from linebot.v3.messaging import MessagingApi' and 'MessagingApi(...).set_default_rich_menu(...)' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api.set_default_rich_menu(rich_menu_id)


In [9]:
# 刪除所有已建立的 Rich Menu
menus = line_bot_api.get_rich_menu_list()
for m in menus:
    print(f"刪除 Rich Menu: {m.rich_menu_id}")
    line_bot_api.delete_rich_menu(m.rich_menu_id)

<ipython-input-9-1560ba7184c9>:2: LineBotSdkDeprecatedIn30: Call to deprecated method get_rich_menu_list. (Use 'from linebot.v3.messaging import MessagingApi' and 'MessagingApi(...).get_rich_menu_list(...)' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  menus = line_bot_api.get_rich_menu_list()


刪除 Rich Menu: richmenu-38cac8dd3bceb99e385e0aab5386e99a


<ipython-input-9-1560ba7184c9>:5: LineBotSdkDeprecatedIn30: Call to deprecated method delete_rich_menu. (Use 'from linebot.v3.messaging import MessagingApi' and 'MessagingApi(...).delete_rich_menu(...)' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api.delete_rich_menu(m.rich_menu_id)


In [11]:
from flask import Flask, request, abort
# for  colab
from pyngrok import ngrok
from flask_ngrok import run_with_ngrok
from google.colab import userdata
# for line bot
from linebot import (
    LineBotApi, WebhookHandler
)
from linebot.exceptions import (
    InvalidSignatureError
)
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage, MessageAction,
    CameraAction, CameraRollAction, LocationAction,
    QuickReply, QuickReplyButton
)

def ngrok_start():
    ngrok.set_auth_token(userdata.get('NGROK_AUTHTOKEN'))
    public_url = ngrok.connect(5000)
    run_with_ngrok(app)

app = Flask(__name__)
ngrok_start()

line_bot_api = LineBotApi(userdata.get('LINE_CHANNEL_ACCESS_TOKEN'))
handler = WebhookHandler(userdata.get('LINE_CHANNEL_SECRET'))

@app.route("/callback", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        print("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)
    return 'OK'

@handler.add(MessageEvent, message=TextMessage)
def handle_text(event):
    # confirm template
    if event.message.text == '圖文選單：我要傳位置':
        reply = TextSendMessage(
            text="請分享您的位置",
            quick_reply=QuickReply(
                items=[QuickReplyButton(action=LocationAction(label="傳送位置"))]
            )
        )
    elif event.message.text == '圖文選單：我要拍照':
        reply = TextSendMessage(
            text="請拍照",
            quick_reply=QuickReply(
                items=[QuickReplyButton(action=CameraAction(label="立即拍照"))]
            )
        )
    elif event.message.text == '圖文選單：選擇照片':
        reply = TextSendMessage(
            text="選擇照片",
            quick_reply=QuickReply(
                items=[QuickReplyButton(action=CameraRollAction(label="選擇照片"))]
            )
        )
    else:
        reply = TextSendMessage(text=event.message.text)
    line_bot_api.reply_message(event.reply_token, reply)

# 啟動 web server
app.run()

<ipython-input-11-88e0f9213a0f>:27: LineBotSdkDeprecatedIn30: Call to deprecated class LineBotApi. (Use v3 class; linebot.v3.<feature>. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api = LineBotApi(userdata.get('LINE_CHANNEL_ACCESS_TOKEN'))


 * Serving Flask app '__main__'
 * Debug mode: off


<ipython-input-11-88e0f9213a0f>:28: LineBotSdkDeprecatedIn30: Call to deprecated class WebhookHandler. (Use 'from linebot.v3.webhook import WebhookHandler' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  handler = WebhookHandler(userdata.get('LINE_CHANNEL_SECRET'))
INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://5e03-34-31-36-33.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [19/Apr/2025 14:50:49] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [19/Apr/2025 14:50:50] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [19/Apr/2025 14:51:08] "POST /callback HTTP/1.1" 200 -
<ipython-input-11-88e0f9213a0f>:67: LineBotSdkDeprecatedIn30: Call to deprecated method reply_message. (Use 'from linebot.v3.messaging import MessagingApi' and 'MessagingApi(...).reply_message(...)' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api.reply_message(event.reply_token, reply)
INFO:werkzeug:127.0.0.1 - - [19/Apr/2025 14:51:21] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [19/Apr/2025 14:51:25] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [19/Apr/2025 14:51:32] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [19/Apr/2025 14:51:35] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [19/Apr/20